In [1]:
import geoai

test_image = "G:\\data\\test\\image\\tile_000002.tif"
model_path = "G:\\data\\models\\best_model.pth"
masks_path = "G:\\data\\models\\test1.tif"

In [2]:
output_vector_path = "G:\\data\\models\\prediction.geojson"
gdf = geoai.orthogonalize(masks_path, output_vector_path, epsilon=2)




Processing 83 features...


Converting features: 100%|██████████| 83/83 [00:00<00:00, 27195.32shape/s]

Saving to G:\data\models\prediction.geojson...
Done!


In [ ]:
import leafmap.foliumap as leafmap # Using 'foliumap' bypasses the VS Code widget bug!
import geopandas as gpd
# ... your other imports ...

# 1. Generate your GeoDataFrame (assuming this still works fine)
# gdf = geoai.orthogonalize(masks_path, output_vector_path, epsilon=2)

# 2. Initialize a pure HTML map
m = leafmap.Map()

# 3. Add your local raster image as the bottom layer
m.add_raster(test_image, layer_name=masks_path)

# 4. Add your GeoDataFrame (the masks) on top
m.add_gdf(
    gdf, 
    layer_name=masks_path, 
    style={"color": "red", "fillOpacity": 0.2, "weight": 2}
)

# 5. Display the map (it will have a layer control box in the top right)
m